<a href="https://colab.research.google.com/github/cuongdaoo/Normalization-of-Property-Data-Using-Edit-Distance-and-Word-Embedding/blob/main/INTERNSHIP_PROJECT_ON_STANDARDIZING_HOUSE_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORTING ESSENTIAL LIBRABIES

In [ ]:
!pip install emoji
!pip install pandasql
!pip install sentence-transformers
!pip install nltk spacy numpy
!python -m spacy download en_core_web_sm
!pip install git+https://github.com/boudinfl/pke.git
!pip install unidecode


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26772 sha256=da14086c24c7e9a6efac61bf06d0137e39eea1cd7c30e9f540764447c4e1dc08
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-s6aqvaeo
  Running comman

In [ ]:
import emoji
import string
import numpy as np
import pandas as pd
import pandasql as ps
import unidecode

# Natural Language Processing packages
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import re
import nltk
from nltk.metrics import edit_distance
import torch
import torch.nn.functional as F

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#IMPORTING DATA

In [ ]:
data_street_training=pd.read_csv('data_input_output/ten_duong.csv')
data_address=pd.read_csv('data_input_output/dia_chinh_v3.csv')

In [ ]:
data_street_training

,Street_ID,Street_Name
0,1,Đặng Văn Ngữ
1,2,Nguyễn Văn Cừ
2,3,Trần Hưng Đạo
3,4,Phạm Văn Đồng
4,5,Nguyễn Tri Phương
...,...,...
95,96,Phan Đăng Lưu
96,97,Huỳnh Văn Nghệ
97,98,Trương Quốc Dung
98,99,Nguyễn Hữu Thọ


In [ ]:
data_address

,Street_ID,Street_Name,Error_Type
0,1,"SN 19C, Đặng Văn Ngữ",không lỗi
1,1,"SN 364, Đặng ăVn Ngữ",lỗi đổi thứ tự các từ
2,1,"SN 41B, Ñaëng Vaên Ngöõ",lỗi font
3,1,"SN 526C, Đặng Văn Ngữ",lỗi thiếu từ + đổi thứ tự các từ
4,1,"SN 601, Đặng Văn",lỗi sai chính tả + thiếu từ
...,...,...,...
1095,100,"SN 723C, Ñaëng Vaên Saâm",lỗi font
1096,100,"SN 810/57, Nhà trống Đặng Văn Sâm",lỗi thêm từ gây nhiễu
1097,100,"SN 832, Đặgn Văn Sâm",lỗi đổi thứ tự các từ
1098,100,"SN 939, Đặng Văn Sẩm",lỗi sai chính tả


#CLEANING DATA

In [ ]:
#Copy raw data
data_address['Street_Name_Raw']=data_address['Street_Name']
data_street_training['Street_Name_Raw']=data_street_training['Street_Name']

In [ ]:

def clean(text):
    # Convert a string to uppercase
    text = str(text).upper()
    # Remove characters within square bracket
    text = re.sub('\[.*?\]', '', text)
    # Remove HTML tags
    text = re.sub('https?://\S+|www\.\S+', '', text)
    # Remove web URLs
    text = re.sub('<.*?>+', '', text)
    # Remove punctuation marks
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # Remove newline characters
    text = re.sub('\n', '', text)
    # Remove_vietnamese_accents
    text= unidecode.unidecode(text)

    # Remove emoji
    text = emoji.replace_emoji(text)
    return text

In [ ]:
data_address['Street_Name']=data_address['Street_Name'].apply(lambda x:clean(x))
data_street_training['Street_Name']=data_street_training['Street_Name'].apply(lambda x:clean(x))

In [ ]:
data_address

,Street_ID,Street_Name,Error_Type,Street_Name_Raw
0,1,SN 19C DANG VAN NGU,không lỗi,"SN 19C, Đặng Văn Ngữ"
1,1,SN 364 DANG AVN NGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ"
2,1,SN 41B NAENG VAEN NGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ"
3,1,SN 526C DANG VAN NGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ"
4,1,SN 601 DANG VAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn"
...,...,...,...,...
1095,100,SN 723C NAENG VAEN SAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm"
1096,100,SN 81057 NHA TRONG DANG VAN SAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm"
1097,100,SN 832 DAGN VAN SAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm"
1098,100,SN 939 DANG VAN SAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm"


In [ ]:
data_street_training

,Street_ID,Street_Name,Street_Name_Raw
0,1,DANG VAN NGU,Đặng Văn Ngữ
1,2,NGUYEN VAN CU,Nguyễn Văn Cừ
2,3,TRAN HUNG DAO,Trần Hưng Đạo
3,4,PHAM VAN DONG,Phạm Văn Đồng
4,5,NGUYEN TRI PHUONG,Nguyễn Tri Phương
...,...,...,...
95,96,PHAN DANG LUU,Phan Đăng Lưu
96,97,HUYNH VAN NGHE,Huỳnh Văn Nghệ
97,98,TRUONG QUOC DUNG,Trương Quốc Dung
98,99,NGUYEN HUU THO,Nguyễn Hữu Thọ


In [ ]:
data_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Street_ID        1100 non-null   int64 
 1   Street_Name      1100 non-null   object
 2   Error_Type       1100 non-null   object
 3   Street_Name_Raw  1100 non-null   object
dtypes: int64(1), object(3)
memory usage: 34.5+ KB


In [ ]:
data_street_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Street_ID        100 non-null    int64 
 1   Street_Name      100 non-null    object
 2   Street_Name_Raw  100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


#EDIT DISTANCE


In [ ]:
e_data_training=data_street_training.copy()
e_data_address=data_address.copy()

In [ ]:
e_data_training['Street_Name'] = e_data_training['Street_Name'].str.replace(r'\s+', '', regex=True)
e_data_training['Street_Name'] = e_data_training['Street_Name'].str.upper()
e_data_address['Street_Name'] = e_data_address['Street_Name'].str.replace(r'\s+', '', regex=True)
e_data_address['Street_Name'] = e_data_address['Street_Name'].str.upper()

In [ ]:
e_data_training

,Street_ID,Street_Name,Street_Name_Raw
0,1,DANGVANNGU,Đặng Văn Ngữ
1,2,NGUYENVANCU,Nguyễn Văn Cừ
2,3,TRANHUNGDAO,Trần Hưng Đạo
3,4,PHAMVANDONG,Phạm Văn Đồng
4,5,NGUYENTRIPHUONG,Nguyễn Tri Phương
...,...,...,...
95,96,PHANDANGLUU,Phan Đăng Lưu
96,97,HUYNHVANNGHE,Huỳnh Văn Nghệ
97,98,TRUONGQUOCDUNG,Trương Quốc Dung
98,99,NGUYENHUUTHO,Nguyễn Hữu Thọ


In [ ]:
e_data_address

,Street_ID,Street_Name,Error_Type,Street_Name_Raw
0,1,SN19CDANGVANNGU,không lỗi,"SN 19C, Đặng Văn Ngữ"
1,1,SN364DANGAVNNGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ"
2,1,SN41BNAENGVAENNGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ"
3,1,SN526CDANGVANNGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ"
4,1,SN601DANGVAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn"
...,...,...,...,...
1095,100,SN723CNAENGVAENSAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm"
1096,100,SN81057NHATRONGDANGVANSAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm"
1097,100,SN832DAGNVANSAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm"
1098,100,SN939DANGVANSAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm"


In [ ]:
#Create a dataframe to contain the calculation data
df_edit=pd.DataFrame(columns=['Street_ID_Add','Street_Name_Add','Street_ID_Train','Street_Name_Train','Edit_distance'])

In [ ]:
for add_idx, add_row in e_data_address.iterrows():
    for street_idx, street_row in e_data_training.iterrows():
        # Calculate the edit distance between fb_username and ig_username
        distance = edit_distance(add_row['Street_Name'], street_row['Street_Name'])

        # Create a new DataFrame for the current row
        new_row = pd.DataFrame({
              'Street_ID_Add': [add_row['Street_ID']],
              'Street_Name_Add': [add_row['Street_Name']],
              'Street_Name_Add_Raw': [add_row['Street_Name_Raw']],
              'Street_ID_Train': [street_row['Street_ID']],
              'Street_Name_Train': [street_row['Street_Name']],
              'Street_Name_Train_Raw': [street_row['Street_Name_Raw']],
              'Edit_distance': [distance]
          })

        # Use concat to add the new row to df_edit
        df_edit = pd.concat([df_edit, new_row], ignore_index=True)



In [ ]:
df_edit

,Street_ID_Add,Street_Name_Add,Street_ID_Train,Street_Name_Train,Edit_distance,Street_Name_Add_Raw,Street_Name_Train_Raw
0,1,SN19CDANGVANNGU,1,DANGVANNGU,5,"SN 19C, Đặng Văn Ngữ",Đặng Văn Ngữ
1,1,SN19CDANGVANNGU,2,NGUYENVANCU,9,"SN 19C, Đặng Văn Ngữ",Nguyễn Văn Cừ
2,1,SN19CDANGVANNGU,3,TRANHUNGDAO,12,"SN 19C, Đặng Văn Ngữ",Trần Hưng Đạo
3,1,SN19CDANGVANNGU,4,PHAMVANDONG,11,"SN 19C, Đặng Văn Ngữ",Phạm Văn Đồng
4,1,SN19CDANGVANNGU,5,NGUYENTRIPHUONG,14,"SN 19C, Đặng Văn Ngữ",Nguyễn Tri Phương
...,...,...,...,...,...,...,...
109995,100,SN97257ADANGVANSAM,96,PHANDANGLUU,14,"SN 972/57A, Đặng Văn Sâm",Phan Đăng Lưu
109996,100,SN97257ADANGVANSAM,97,HUYNHVANNGHE,15,"SN 972/57A, Đặng Văn Sâm",Huỳnh Văn Nghệ
109997,100,SN97257ADANGVANSAM,98,TRUONGQUOCDUNG,17,"SN 972/57A, Đặng Văn Sâm",Trương Quốc Dung
109998,100,SN97257ADANGVANSAM,99,NGUYENHUUTHO,16,"SN 972/57A, Đặng Văn Sâm",Nguyễn Hữu Thọ


In [ ]:
#df_edit.to_csv('edit_distance.csv', encoding='utf-8-sig')

##ACCURACY

In [ ]:
# Create DataFrame e_accuracy
e_accuracy = df_edit.copy()
# Create column 'min_edit_distance' (select the smallest edit_distance)
e_accuracy['min_edit_distance'] = 0
for Street_Name_Add in e_accuracy['Street_Name_Add'].unique():
    min_distance = e_accuracy[e_accuracy['Street_Name_Add'] == Street_Name_Add]['Edit_distance'].min()
    e_accuracy.loc[(e_accuracy['Street_Name_Add'] == Street_Name_Add) & (e_accuracy['Edit_distance'] == min_distance), 'min_edit_distance'] = 1
# Mark 1 if 'Street_ID_Add' equals 'edit_distance' and 'min_edit_distance_random' = 1
e_accuracy['addressid_equals_edit_dist'] = 0
e_accuracy.loc[(e_accuracy['Street_ID_Add'] == e_accuracy['Street_ID_Train']) & (e_accuracy['min_edit_distance'] == 1), 'addressid_equals_edit_dist'] = 1

In [ ]:
result_edit= e_accuracy['addressid_equals_edit_dist'].sum()/1100*100
print("Accuracy: ",result_edit, " %")

Accuracy:  95.0909090909091  %


Output

In [ ]:
a1=e_accuracy[(e_accuracy['min_edit_distance'] == 1)&(e_accuracy['Street_ID_Add']==e_accuracy['Street_ID_Train'])]
a2=e_accuracy[(e_accuracy['min_edit_distance'] == 0)&(e_accuracy['Street_ID_Add']==e_accuracy['Street_ID_Train'])]
a = pd.concat([a1, a2], ignore_index=True)
# sort by Street_ID_Add
a = a.sort_values(by=['Street_ID_Add'])
a

,Street_ID_Add,Street_Name_Add,Street_ID_Train,Street_Name_Train,Edit_distance,Street_Name_Add_Raw,Street_Name_Train_Raw,min_edit_distance,addressid_equals_edit_dist
0,1,SN19CDANGVANNGU,1,DANGVANNGU,5,"SN 19C, Đặng Văn Ngữ",Đặng Văn Ngữ,1,1
1,1,SN364DANGAVNNGU,1,DANGVANNGU,7,"SN 364, Đặng ăVn Ngữ",Đặng Văn Ngữ,1,1
2,1,SN41BNAENGVAENNGOO,1,DANGVANNGU,10,"SN 41B, Ñaëng Vaên Ngöõ",Đặng Văn Ngữ,1,1
3,1,SN526CDANGVANNGU,1,DANGVANNGU,6,"SN 526C, Đặng Văn Ngữ",Đặng Văn Ngữ,1,1
4,1,SN601DANGVAN,1,DANGVANNGU,8,"SN 601, Đặng Văn",Đặng Văn Ngữ,1,1
...,...,...,...,...,...,...,...,...,...
1039,100,SN593DANGSAM,100,DANGVANSAM,7,"SN 593, Đặng Sâm",Đặng Văn Sâm,1,1
1038,100,SN342ADNGVANSAM,100,DANGVANSAM,6,"SN 342, ặĐng Văn Sâm",Đặng Văn Sâm,1,1
1037,100,SN26427NHA2TANGDANGVANSAM,100,DANGVANSAM,15,"SN 264/27, Nhà 2 tầng Đặng Văn Sậm",Đặng Văn Sâm,1,1
1035,100,SN21712DANGSAM,100,DANGVANSAM,9,"SN 217/12, Đặng Sẩm",Đặng Văn Sâm,1,1


In [ ]:
a.to_csv('/content/data_input_output/edit_distance_output.csv', encoding='utf-8-sig')

#Word embedding

##E5-small-V2 MODEL

Copy data

In [ ]:
e5_data_add=data_address.copy()
e5_data_training=data_street_training.copy()

In [ ]:
e5_data_add

,Street_ID,Street_Name,Error_Type,Street_Name_Raw
0,1,SN 19C DANG VAN NGU,không lỗi,"SN 19C, Đặng Văn Ngữ"
1,1,SN 364 DANG AVN NGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ"
2,1,SN 41B NAENG VAEN NGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ"
3,1,SN 526C DANG VAN NGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ"
4,1,SN 601 DANG VAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn"
...,...,...,...,...
1095,100,SN 723C NAENG VAEN SAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm"
1096,100,SN 81057 NHA TRONG DANG VAN SAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm"
1097,100,SN 832 DAGN VAN SAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm"
1098,100,SN 939 DANG VAN SAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm"


In [ ]:
e5_data_training

,Street_ID,Street_Name,Street_Name_Raw
0,1,DANG VAN NGU,Đặng Văn Ngữ
1,2,NGUYEN VAN CU,Nguyễn Văn Cừ
2,3,TRAN HUNG DAO,Trần Hưng Đạo
3,4,PHAM VAN DONG,Phạm Văn Đồng
4,5,NGUYEN TRI PHUONG,Nguyễn Tri Phương
...,...,...,...
95,96,PHAN DANG LUU,Phan Đăng Lưu
96,97,HUYNH VAN NGHE,Huỳnh Văn Nghệ
97,98,TRUONG QUOC DUNG,Trương Quốc Dung
98,99,NGUYEN HUU THO,Nguyễn Hữu Thọ


###MODELING

In [ ]:
we_e5_model= SentenceTransformer('intfloat/e5-small-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
e5_add_list=e5_data_add['Street_Name'].tolist()
e5_training_list=e5_data_training['Street_Name'].tolist()
e5_embeddings_add = we_e5_model.encode(e5_add_list, convert_to_tensor=True)
e5_embeddings_training = we_e5_model.encode(e5_training_list, convert_to_tensor=True)
#convert to numpy array for convenient storage
e5_embeddings_add = e5_embeddings_add.detach().numpy()
e5_embeddings_training = e5_embeddings_training.detach().numpy()
#convert the embedding vectors representing the keywords into lists to store into the corresponding "text_vector" columns in the two DataFrames
e5_data_add['text_vector'] = e5_embeddings_add.tolist()
e5_data_training['text_vector'] = e5_embeddings_training.tolist()


In [ ]:
e5_data_add

,Street_ID,Street_Name,Error_Type,Street_Name_Raw,text_vector
0,1,SN 19C DANG VAN NGU,không lỗi,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0..."
1,1,SN 364 DANG AVN NGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ","[-0.09630431234836578, 0.04084962233901024, 0...."
2,1,SN 41B NAENG VAEN NGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ","[-0.11035669595003128, 0.06765998899936676, 0...."
3,1,SN 526C DANG VAN NGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ","[-0.11222200095653534, 0.05837829038500786, 0...."
4,1,SN 601 DANG VAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn","[-0.1345066875219345, 0.03675467148423195, 0.0..."
...,...,...,...,...,...
1095,100,SN 723C NAENG VAEN SAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm","[-0.10876761376857758, 0.04505181312561035, 0...."
1096,100,SN 81057 NHA TRONG DANG VAN SAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm","[-0.10271857678890228, 0.06613828241825104, 0...."
1097,100,SN 832 DAGN VAN SAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm","[-0.10491720587015152, 0.043730538338422775, 0..."
1098,100,SN 939 DANG VAN SAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm","[-0.09822916984558105, 0.030959758907556534, 0..."


In [ ]:
e5_data_training

,Street_ID,Street_Name,Street_Name_Raw,text_vector
0,1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02..."
1,2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.08101893961429596, 0.0592430979013443, 0.0..."
2,3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.07119498401880264, 0.08053550124168396, 0...."
3,4,PHAM VAN DONG,Phạm Văn Đồng,"[-0.10132669657468796, 0.08450060337781906, 0...."
4,5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.04958999529480934, 0.08231433480978012, 0...."
...,...,...,...,...
95,96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.09488974511623383, 0.048026930540800095, 0..."
96,97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.07475990802049637, 0.0443415492773056, 0.0..."
97,98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.06661833822727203, 0.07228459417819977, 0...."
98,99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.0718488022685051, 0.06744851171970367, 0.0..."


In [ ]:
# calculate cosine similarity between 2 data
data_e5_cos = []
for add in range(len(e5_data_add)):
    for training in range(len(e5_data_training)):
            cosine = F.cosine_similarity(torch.tensor(e5_data_add["text_vector"][add]), torch.tensor(e5_data_training["text_vector"][training]), dim=-1).item()
            data_e5_cos.append({
                "Street_ID_Add": e5_data_add["Street_ID"][add],
                "Street_Name_Add": e5_data_add["Street_Name"][add],
                'Street_Name_Add_Raw': e5_data_add["Street_Name_Raw"][add],
                "text_vector_address": e5_data_add["text_vector"][add],
                "Street_ID_Train": e5_data_training["Street_ID"][training],
                "Street_Name_Train": e5_data_training["Street_Name"][training],
                'Street_Name_Train_Raw': e5_data_training["Street_Name_Raw"][training],
                "text_vector_training": e5_data_training["text_vector"][training],
                "cosine": cosine
            })




In [ ]:
data_e5_cos=pd.DataFrame(data_e5_cos)
data_e5_cos

,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.915395
1,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.08101893961429596, 0.0592430979013443, 0.0...",0.809017
2,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.07119498401880264, 0.08053550124168396, 0....",0.785746
3,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",4,PHAM VAN DONG,Phạm Văn Đồng,"[-0.10132669657468796, 0.08450060337781906, 0....",0.816895
4,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.04958999529480934, 0.08231433480978012, 0....",0.759713
...,...,...,...,...,...,...,...,...,...
109995,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.09488974511623383, 0.048026930540800095, 0...",0.827686
109996,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.07475990802049637, 0.0443415492773056, 0.0...",0.762440
109997,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.06661833822727203, 0.07228459417819977, 0....",0.792728
109998,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.0718488022685051, 0.06744851171970367, 0.0...",0.768609


###ACCURACY

In [ ]:
e5_accuracy=data_e5_cos.copy()
# Create a 'max_cosine' column (pick out the largest cosine)
e5_accuracy['max_cosine'] = 0
for Street_Name_Add in e5_accuracy['Street_Name_Add'].unique():
    max_cosine = e5_accuracy[e5_accuracy['Street_Name_Add'] == Street_Name_Add]['cosine'].max()
    e5_accuracy.loc[(e5_accuracy['Street_Name_Add'] == Street_Name_Add) & (e5_accuracy['cosine'] == max_cosine), 'max_cosine'] = 1

# Mark 1 if 'add_userid' equals 'training_userid' and 'max_cosine_random' = 1
e5_accuracy['equals_id'] = 0
e5_accuracy.loc[(e5_accuracy['Street_ID_Add'] == e5_accuracy['Street_ID_Train']) & (e5_accuracy['max_cosine'] == 1), 'equals_id'] = 1
e5_accuracy


,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine,max_cosine,equals_id
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.915395,1,1
1,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.08101893961429596, 0.0592430979013443, 0.0...",0.809017,0,0
2,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.07119498401880264, 0.08053550124168396, 0....",0.785746,0,0
3,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",4,PHAM VAN DONG,Phạm Văn Đồng,"[-0.10132669657468796, 0.08450060337781906, 0....",0.816895,0,0
4,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.04958999529480934, 0.08231433480978012, 0....",0.759713,0,0
...,...,...,...,...,...,...,...,...,...,...,...
109995,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.09488974511623383, 0.048026930540800095, 0...",0.827686,0,0
109996,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.07475990802049637, 0.0443415492773056, 0.0...",0.762440,0,0
109997,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.06661833822727203, 0.07228459417819977, 0....",0.792728,0,0
109998,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[-0.06682023406028748, 0.04141850396990776, 0....",99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.0718488022685051, 0.06744851171970367, 0.0...",0.768609,0,0


In [ ]:
# print table where equals_id=1 and max_cosine=1
a=e5_accuracy[(e5_accuracy['max_cosine'] == 1)&(e5_accuracy['equals_id'] == 1)]
a


,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine,max_cosine,equals_id
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.10614337772130966, 0.030227424576878548, 0...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.915395,1,1
100,1,SN 364 DANG AVN NGU,"SN 364, Đặng ăVn Ngữ","[-0.09630431234836578, 0.04084962233901024, 0....",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.868838,1,1
300,1,SN 526C DANG VAN NGU,"SN 526C, Đặng Văn Ngữ","[-0.11222200095653534, 0.05837829038500786, 0....",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.896344,1,1
500,1,SN 68350 DANG VAN NGU,"SN 683/50, Đặng Văn Ngữ","[-0.10004647821187973, 0.02371145226061344, 0....",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.901588,1,1
600,1,SN 69569 KHONG XAC DINH DANG VAN NGU,"SN 695/69, Không xác định Đặng Văn Ngữ","[-0.10896216332912445, 0.016295472159981728, 0...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.10248818248510361, 0.073002390563488, 0.02...",0.861359,1,1
...,...,...,...,...,...,...,...,...,...,...,...
109499,100,SN 693 DANG VAN SAM,"SN 693, Đặng Văn Sâm","[-0.10973304510116577, 0.016407260671257973, 0...",100,DANG VAN SAM,Đặng Văn Sâm,"[-0.06774939596652985, 0.05358193442225456, 0....",0.919013,1,1
109699,100,SN 81057 NHA TRONG DANG VAN SAM,"SN 810/57, Nhà trống Đặng Văn Sâm","[-0.10271857678890228, 0.06613828241825104, 0....",100,DANG VAN SAM,Đặng Văn Sâm,"[-0.06774939596652985, 0.05358193442225456, 0....",0.867965,1,1
109799,100,SN 832 DAGN VAN SAM,"SN 832, Đặgn Văn Sâm","[-0.10491720587015152, 0.043730538338422775, 0...",100,DANG VAN SAM,Đặng Văn Sâm,"[-0.06774939596652985, 0.05358193442225456, 0....",0.879223,1,1
109899,100,SN 939 DANG VAN SAM,"SN 939, Đặng Văn Sẩm","[-0.09822916984558105, 0.030959758907556534, 0...",100,DANG VAN SAM,Đặng Văn Sâm,"[-0.06774939596652985, 0.05358193442225456, 0....",0.905510,1,1


In [ ]:
result_e5= e5_accuracy['equals_id'].sum()/1100*100
print("Acurracy: ",result_e5, " %")

Acurracy:  86.0909090909091  %


##GTE-LARGE


Copy data

In [ ]:
gte_data_add=data_address.copy()
gte_data_training=data_street_training.copy()

###MODELING

In [ ]:
gte_model = SentenceTransformer('thenlper/gte-large')

In [ ]:
add_gte_list=gte_data_add['Street_Name'].tolist()
training_gte_list=gte_data_training['Street_Name'].tolist()
gte_embeddings_add = gte_model.encode(add_gte_list, convert_to_tensor=True)
gte_embeddings_training = gte_model.encode(training_gte_list, convert_to_tensor=True)
gte_embeddings_add = gte_embeddings_add.detach().numpy()
gte_embeddings_training = gte_embeddings_training.detach().numpy()
gte_data_add['text_vector'] = gte_embeddings_add.tolist()
gte_data_training['text_vector'] = gte_embeddings_training.tolist()


In [ ]:
gte_data_add

,Street_ID,Street_Name,Error_Type,Street_Name_Raw,text_vector
0,1,SN 19C DANG VAN NGU,không lỗi,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,..."
1,1,SN 364 DANG AVN NGU,lỗi đổi thứ tự các từ,"SN 364, Đặng ăVn Ngữ","[0.004170349799096584, -0.011334676295518875, ..."
2,1,SN 41B NAENG VAEN NGOO,lỗi font,"SN 41B, Ñaëng Vaên Ngöõ","[-0.002184184966608882, 0.00436954153701663, -..."
3,1,SN 526C DANG VAN NGU,lỗi thiếu từ + đổi thứ tự các từ,"SN 526C, Đặng Văn Ngữ","[-0.006716499570757151, -0.009630143642425537,..."
4,1,SN 601 DANG VAN,lỗi sai chính tả + thiếu từ,"SN 601, Đặng Văn","[-0.002141285687685013, 0.008678363636136055, ..."
...,...,...,...,...,...
1095,100,SN 723C NAENG VAEN SAAM,lỗi font,"SN 723C, Ñaëng Vaên Saâm","[-0.006809805985540152, 0.011011254042387009, ..."
1096,100,SN 81057 NHA TRONG DANG VAN SAM,lỗi thêm từ gây nhiễu,"SN 810/57, Nhà trống Đặng Văn Sâm","[-0.01720387302339077, 0.016107188537716866, -..."
1097,100,SN 832 DAGN VAN SAM,lỗi đổi thứ tự các từ,"SN 832, Đặgn Văn Sâm","[0.0058784205466508865, 0.02194984257221222, 0..."
1098,100,SN 939 DANG VAN SAM,lỗi sai chính tả,"SN 939, Đặng Văn Sẩm","[0.02839365229010582, 0.025042040273547173, -0..."


In [ ]:
gte_data_training

,Street_ID,Street_Name,Street_Name_Raw,text_vector
0,1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,..."
1,2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.005895429756492376, 0.005340247415006161, ..."
2,3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.028865694999694824, -0.038922276347875595,..."
3,4,PHAM VAN DONG,Phạm Văn Đồng,"[0.0032692996319383383, -0.014205255545675755,..."
4,5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.03897061198949814, -0.013363678008317947, ..."
...,...,...,...,...
95,96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.024394823238253593, -0.023049095645546913,..."
96,97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.012660451233386993, -0.004363514017313719,..."
97,98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.016411006450653076, -0.01799963042140007, ..."
98,99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.004869547672569752, 0.0011099106632173061,..."


In [ ]:
# calculate cosine similarity between 2 data
data_gte_cos = []
for add in range(len(gte_data_add)):
    for training in range(len(gte_data_training)):
            cosine = F.cosine_similarity(torch.tensor(gte_data_add["text_vector"][add]), torch.tensor(gte_data_training["text_vector"][training]), dim=-1).item()
            data_gte_cos.append({
                "Street_ID_Add": gte_data_add["Street_ID"][add],
                "Street_Name_Add": gte_data_add["Street_Name"][add],
                "Street_Name_Add_Raw": gte_data_add["Street_Name_Raw"][add],
                "text_vector_address": gte_data_add["text_vector"][add],
                "Street_ID_Train": gte_data_training["Street_ID"][training],
                "Street_Name_Train": gte_data_training["Street_Name"][training],
                "Street_Name_Train_Raw": gte_data_training["Street_Name_Raw"][training],
                "text_vector_training": gte_data_training["text_vector"][training],
                "cosine": cosine
            })



In [ ]:
data_gte_cos=pd.DataFrame(data_gte_cos)
data_gte_cos

,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.920342
1,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.005895429756492376, 0.005340247415006161, ...",0.851131
2,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.028865694999694824, -0.038922276347875595,...",0.797677
3,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",4,PHAM VAN DONG,Phạm Văn Đồng,"[0.0032692996319383383, -0.014205255545675755,...",0.815442
4,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.03897061198949814, -0.013363678008317947, ...",0.823922
...,...,...,...,...,...,...,...,...,...
109995,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.024394823238253593, -0.023049095645546913,...",0.833848
109996,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.012660451233386993, -0.004363514017313719,...",0.790913
109997,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.016411006450653076, -0.01799963042140007, ...",0.790205
109998,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.004869547672569752, 0.0011099106632173061,...",0.799008


###ACCURACY

In [ ]:
gte_accuracy=data_gte_cos.copy()
# Create a 'max_cosine' column (pick out the largest cosine)
gte_accuracy['max_cosine'] = 0
for Street_Name_Add in gte_accuracy['Street_Name_Add'].unique():
    max_cosine = gte_accuracy[gte_accuracy['Street_Name_Add'] == Street_Name_Add]['cosine'].max()
    gte_accuracy.loc[(gte_accuracy['Street_Name_Add'] == Street_Name_Add) & (gte_accuracy['cosine'] == max_cosine), 'max_cosine'] = 1

# Mark 1 if 'add_userid' equals 'training_userid' and 'max_cosine_random' = 1
gte_accuracy['equals_id'] = 0
gte_accuracy.loc[(gte_accuracy['Street_ID_Add'] == gte_accuracy['Street_ID_Train']) & (gte_accuracy['max_cosine'] == 1), 'equals_id'] = 1
gte_accuracy


,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine,max_cosine,equals_id
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.920342,1,1
1,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",2,NGUYEN VAN CU,Nguyễn Văn Cừ,"[-0.005895429756492376, 0.005340247415006161, ...",0.851131,0,0
2,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",3,TRAN HUNG DAO,Trần Hưng Đạo,"[-0.028865694999694824, -0.038922276347875595,...",0.797677,0,0
3,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",4,PHAM VAN DONG,Phạm Văn Đồng,"[0.0032692996319383383, -0.014205255545675755,...",0.815442,0,0
4,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",5,NGUYEN TRI PHUONG,Nguyễn Tri Phương,"[-0.03897061198949814, -0.013363678008317947, ...",0.823922,0,0
...,...,...,...,...,...,...,...,...,...,...,...
109995,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",96,PHAN DANG LUU,Phan Đăng Lưu,"[-0.024394823238253593, -0.023049095645546913,...",0.833848,0,0
109996,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",97,HUYNH VAN NGHE,Huỳnh Văn Nghệ,"[-0.012660451233386993, -0.004363514017313719,...",0.790913,0,0
109997,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",98,TRUONG QUOC DUNG,Trương Quốc Dung,"[-0.016411006450653076, -0.01799963042140007, ...",0.790205,0,0
109998,100,SN 97257A DANG VAN SAM,"SN 972/57A, Đặng Văn Sâm","[0.01030462235212326, 0.01216098666191101, -0....",99,NGUYEN HUU THO,Nguyễn Hữu Thọ,"[-0.004869547672569752, 0.0011099106632173061,...",0.799008,0,0


In [ ]:
a=gte_accuracy[(gte_accuracy['max_cosine'] == 1)&(gte_accuracy['equals_id'] == 1)]
a

,Street_ID_Add,Street_Name_Add,Street_Name_Add_Raw,text_vector_address,Street_ID_Train,Street_Name_Train,Street_Name_Train_Raw,text_vector_training,cosine,max_cosine,equals_id
0,1,SN 19C DANG VAN NGU,"SN 19C, Đặng Văn Ngữ","[-0.011203194968402386, -0.008828266523778439,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.920342,1,1
100,1,SN 364 DANG AVN NGU,"SN 364, Đặng ăVn Ngữ","[0.004170349799096584, -0.011334676295518875, ...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.866095,1,1
300,1,SN 526C DANG VAN NGU,"SN 526C, Đặng Văn Ngữ","[-0.006716499570757151, -0.009630143642425537,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.894427,1,1
500,1,SN 68350 DANG VAN NGU,"SN 683/50, Đặng Văn Ngữ","[-0.0010878759203478694, -0.014489724300801754...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.914009,1,1
600,1,SN 69569 KHONG XAC DINH DANG VAN NGU,"SN 695/69, Không xác định Đặng Văn Ngữ","[-9.865411993814632e-05, 0.009861091151833534,...",1,DANG VAN NGU,Đặng Văn Ngữ,"[-0.007941612973809242, -0.020616762340068817,...",0.860435,1,1
...,...,...,...,...,...,...,...,...,...,...,...
109499,100,SN 693 DANG VAN SAM,"SN 693, Đặng Văn Sâm","[0.017218008637428284, 0.015009158290922642, -...",100,DANG VAN SAM,Đặng Văn Sâm,"[0.003528919070959091, -0.008171512745320797, ...",0.909962,1,1
109699,100,SN 81057 NHA TRONG DANG VAN SAM,"SN 810/57, Nhà trống Đặng Văn Sâm","[-0.01720387302339077, 0.016107188537716866, -...",100,DANG VAN SAM,Đặng Văn Sâm,"[0.003528919070959091, -0.008171512745320797, ...",0.903749,1,1
109799,100,SN 832 DAGN VAN SAM,"SN 832, Đặgn Văn Sâm","[0.0058784205466508865, 0.02194984257221222, 0...",100,DANG VAN SAM,Đặng Văn Sâm,"[0.003528919070959091, -0.008171512745320797, ...",0.849970,1,1
109899,100,SN 939 DANG VAN SAM,"SN 939, Đặng Văn Sẩm","[0.02839365229010582, 0.025042040273547173, -0...",100,DANG VAN SAM,Đặng Văn Sâm,"[0.003528919070959091, -0.008171512745320797, ...",0.909785,1,1


In [ ]:
result_gte= gte_accuracy['equals_id'].sum()/1100*100
print("Acurracy: ",result_gte, " %")

Acurracy:  88.63636363636364  %


#COMPARE

In [ ]:
df_compare = pd.DataFrame([[result_edit], [result_e5], [result_gte]],
                         index=['Edit distance', 'Word embedding with e5 model', 'Word embedding with gte model'],
                         columns=['Accuracy'])
df_compare

,Accuracy
Edit distance,95.090909
Word embedding with e5 model,86.090909
Word embedding with gte model,88.636364


**Thus:**
* Edit Distance is the best method for identifying street names when small character differences are the primary distinction.
* Word embeddings are less effective in this case, likely because they focus more on semantic relationships, which are less relevant for street names.